In [1]:
import sys
sys.path.append("..")

In [2]:
import wfdb

from raccoon.utils.annotationutils import trigger_points

def read_record(path, sampto):
    record = wfdb.rdrecord(path, sampto=sampto)
    annotation = wfdb.rdann(path, 'atr', sampto=sampto)
    positions, labels = trigger_points(annotation, with_labels=True)
    return record, positions, labels

def read_records(directory, names, sampto):
    records, positions, labels = [], [],  []
    for name in names:
        path = "/".join([directory, name])
        record, pos, labs = read_record(path, sampto)
        records.append(record)
        positions.append(pos)
        labels.append(labs)
    return records, positions, labels

In [3]:
record_dir = "../physionet.org/files/mitdb/1.0.0"
test_path = '/'.join([record_dir, '106'])
train_names = ["100", "101", "102", "103", "104"]
sampto = None

train_records, train_positions, train_labels = read_records(record_dir, train_names, sampto)
test_record, test_positions, test_labels = read_record(test_path, sampto)

In [4]:
from raccoon.detectors import GarciaBerdonesDetector

detector = GarciaBerdonesDetector(
    name = "GarciaBerdones01",
    batch_size = 32,
    window_size = 20,
    epochs = 3)

detector.train(train_records, train_positions)

Using TensorFlow backend.
/home/Jonas.Chromik/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/Jonas.Chromik/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/Jonas.Chromik/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/Jonas.Chromik/.local/lib/python3.7/sit

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
60935/60935 [==============================] - 86s 1ms/step - loss: 0.0100
Epoch 2/3
60935/60935 [==============================] - 88s 1ms/step - loss: 0.0078
Epoch 3/3
60935/60935 [==============================] - 87s 1ms/step - loss: 0.0080


In [5]:
trigger_signal = detector.trigger_signal(test_record)

In [6]:
from raccoon.utils.triggerutils import signal_to_points

detections, certainties = signal_to_points(trigger_signal, with_certainty=True)

In [7]:
import numpy as np

cert_dict = dict(zip(detections, certainties))

abbr = {'N': 'Normal Beat', 'V': 'Premature Ventricular Contraction'}
label_dict = dict(zip(test_positions, [abbr[l] for l in test_labels]))
label_dict[np.nan] = 'False Positive'

In [8]:
from raccoon.utils.evaluationutils import merge
import pandas as pd

df = pd.DataFrame(
    list(merge(test_positions, detections, tolerance=3)),
    columns=["Position", "Detection", "Error"])

df["Certainty"] = df.Detection.map(cert_dict)
df["Label"] = df.Position.map(label_dict)

In [9]:
df

Position  Detection Error  Certainty           Label
0        351.0      351.0    TP   0.998696     Normal Beat
1        724.0      724.0    TP   1.000000     Normal Beat
2       1086.0     1086.0    TP   0.999528     Normal Beat
3       1448.0     1448.0    TP   0.996393     Normal Beat
4       1829.0     1829.0    TP   0.998302     Normal Beat
...        ...        ...   ...        ...             ...
2431       NaN   645397.0    FP   0.998442  False Positive
2432       NaN   646167.0    FP   0.964219  False Positive
2433       NaN   646195.0    FP   0.998833  False Positive
2434       NaN   649327.0    FP   0.988129  False Positive
2435       NaN   649350.0    FP   0.999578  False Positive

[2436 rows x 5 columns]

In [13]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots()
fig.set_size_inches(10, 5)

df.boxplot(
    column='Certainty', by='Label', ax=ax,
    showfliers=False)

ax.set_ylim(0.8,1)
ax.set_xlabel("")
ax.set_title("Certainty in Different Beat Types")
fig.suptitle("")

ax

In [14]:
fig.savefig('../generated/figures/impact_beat_type.pdf', format='pdf', bbox_inches = 'tight', pad_inches = 0.1)

In [15]:
from scipy.stats import mannwhitneyu

print(mannwhitneyu(
    df[df.Label == 'Normal Beat'].Certainty,
    df[df.Label == 'Premature Ventricular Contraction'].Certainty))

print(mannwhitneyu(
    df[df.Label == 'False Positive'].Certainty,
    df[df.Label == 'Premature Ventricular Contraction'].Certainty))

MannwhitneyuResult(statistic=318614.0, pvalue=1.0006281830779208e-10)
MannwhitneyuResult(statistic=42881.0, pvalue=2.252733742355717e-55)
